<a href="https://colab.research.google.com/github/ithalo-teodosio/pratica_ML/blob/main/ML_Q6_Hackaton_Previsao_valor_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instituição: Unifacisa
# Autor: Ithalo Tedósio Nascimento - Data: 21/abr
# Q6 - Previsão do valor de imóveis

**Questão 6 (avançado) - Previsão do Valor de Imóveis**

Uma imobiliária deseja prever o valor de imóveis com base em características como localização, número de quartos, tamanho do terreno, entre outros.

Tarefas:

- Utilize um dataset de preços de imóveis (exemplo: California Housing do Scikit-Learn).
- Aplique técnicas de feature engineering para melhorar o desempenho do modelo.
- Teste diferentes algoritmos de regressão, como Regressão Linear, XGBoost e Redes Neurais Artificiais (ANNs).
- Avalie os modelos com métricas como RMSE e R².

Pergunta: Qual modelo teve menor erro de previsão? Como otimizar ainda mais o desempenho?

In [1]:
# Instalação do XGBoost e o TensorFlow
!pip install xgboost tensorflow --quiet

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set(style="whitegrid")

In [2]:
# Dataset California Housing
data = fetch_california_housing(as_frame=True)
df = data.frame

# Visualizando as primeiras linhas de dados
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
# Criando novas features
df['rooms_per_person'] = df['AveRooms'] / df['Population']
df['bedroom_ratio'] = df['AveBedrms'] / df['AveRooms']
df['income_per_room'] = df['MedInc'] / df['AveRooms']

# Removendo possíveis infinitos (divisão por zero)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Separando X e y
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

In [4]:
# Dividir e padronizar valores
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Regressão linear
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_test_scaled)

lr_rmse = np.sqrt(mean_squared_error(y_test, lr_preds))
lr_r2 = r2_score(y_test, lr_preds)

print(f"📈 Regressão Linear - RMSE: {lr_rmse:.2f}, R²: {lr_r2:.2f}")

📈 Regressão Linear - RMSE: 0.73, R²: 0.60


In [6]:
# XGBoost
xg_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xg_model.fit(X_train, y_train)
xgb_preds = xg_model.predict(X_test)

xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
xgb_r2 = r2_score(y_test, xgb_preds)

print(f"⚡ XGBoost - RMSE: {xgb_rmse:.2f}, R²: {xgb_r2:.2f}")

⚡ XGBoost - RMSE: 0.47, R²: 0.83


In [7]:
# ANN simples com Keras
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compilar e treinar
model.compile(optimizer='adam', loss='mse')
model.fit(X_train_scaled, y_train, epochs=50, verbose=0, batch_size=32)

# Previsão
ann_preds = model.predict(X_test_scaled).flatten()

ann_rmse = np.sqrt(mean_squared_error(y_test, ann_preds))
ann_r2 = r2_score(y_test, ann_preds)

print(f"🧠 Rede Neural - RMSE: {ann_rmse:.2f}, R²: {ann_r2:.2f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
🧠 Rede Neural - RMSE: 0.53, R²: 0.79


In [8]:
results = pd.DataFrame({
    'Modelo': ['Regressão Linear', 'XGBoost', 'Rede Neural'],
    'RMSE': [lr_rmse, xgb_rmse, ann_rmse],
    'R²': [lr_r2, xgb_r2, ann_r2]
})

print("Comparativo de Desempenho:")
print(results.sort_values('RMSE'))


Comparativo de Desempenho:
             Modelo      RMSE        R²
1           XGBoost  0.468714  0.832348
2       Rede Neural  0.529002  0.786446
0  Regressão Linear  0.728249  0.595281


**Resposta da pergunta:**

- Qual modelo teve menor erro?

Geralmente, XGBoost ou Rede Neural apresentam menor RMSE e maior R² que a Regressão Linear.

- Como otimizar ainda mais?

Tuning de hiperparâmetros (usando GridSearchCV, RandomizedSearch, ou Optuna)

Adição de mais features não-lineares

Uso de Regularização (Ridge, Lasso)

Ensembling (combinar modelos)